# **Product recommender system**

Installing the needed modules.

In [1]:
import os
from gpt4all import Embed4All
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
import plotly.express as px

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create a Spark session

In [3]:
spark = SparkSession.builder.appName("ProductRecommenderSystem").getOrCreate()
spark

Loading the dataset

In [4]:
df=spark.read.csv("/content/drive/MyDrive/rec_system/products_dataset.csv",header=True,inferSchema=True, samplingRatio=1)
df.show()

+----------+--------------------+--------------------+
|product_id|               title|         description|
+----------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|
|        P1|Turmode 30 ft. RP...|If you need more ...|
|        P2|Large Tapestry Bo...|Polyester cover r...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|
|        P9|Traditional Silve...|This transitional...|
|       P10|15 in. x 59 in. O...|Its easy to add a...|
|       P11|1 qt. #350F-7 Wil...|BEHR PREMIUM PLUS...|
|       P12|Anthracite Cordle...|BlindsAvenue ligh...|
|       P13|SlimGrip 78-Inch ...|Luverne SlimGrip ...|
|       P14|6 in. x 28 in. x ...|Our Rustic Collec...|
|       P1

List of 8 products recently viewed by the user.

In [5]:
recently_viewed_products = [
    'P316',
    'P333',
    'P1115',
    'P1691',
    'P1082',
    'P397',
    'P1441',
    'P1054',
]

### Preparing the dataset

Combining `title` and `description` Columns

In [6]:
df = df.withColumn("combined_text", concat_ws(" ", "title", "description"))
df.show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|       combined_text|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|
|        P9|Traditional Silve...|This transitional...|Traditional Silve...|
|       P10|

get the combined_text column and convert it into a list

In [7]:
list_combined = df.select("combined_text").rdd.flatMap(lambda x : x).collect()
print(list_combined[:4])

["Men's 3X Large Carbon Heather Cotton/Polyester Rain Defender Paxton Heavyweight Hooded Zip-Front Sweatshirt This heavyweight, water-repellent hooded sweatshirt has a zip front for fast layering. ORIGINAL FIT. 13 oz., 75% cotton/25% polyester blend with Rain Defender durable water repellent. Attached, jersey-lined three-piece hood with drawcord closure. Antique-finish brass front zipper. Two front hand-warmer pockets have a hidden security pocket inside. Stretchable, spandex-reinforced rib-knit cuffs and waistband. Locker loop facilitates hanging.", "Turmode 30 ft. RP TNC Female to RP TNC Male Adapter Cable If you need more length between your existing wireless device and Hi-Gain Antenna, this is the product for you. It's compatible with most Wi-Fi Antennas, so it is easy for you to extend your wireless network. Just replace your existing cable that runs between your wireless device and Antenna and you're ready to use your network with extended range.", 'Large Tapestry Bolster Bed Pol

Use GPT4all text embedding model to create the vector embeddings.

In [8]:
feature_extractor = Embed4All()
train_embeddings=[]

count=0
# Training sample embeddings
for text in list_combined:
  embeddings = feature_extractor.embed(text)
  train_embeddings.append(embeddings)
  count+=1
  if count%1000==0:
    print(count)

1000
2000


Let't put the embedding vectors into our original dataframe

Convert embedding vectors list into a Pyspark DataFrame

In [9]:
features_column_names = [f"embedding_{i}" for i in range(len(train_embeddings[0]))]
embeddings_df = spark.createDataFrame(train_embeddings, schema=features_column_names)
embeddings_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

Add unique `row_id` to each row in the pysaprk dataframe

In [10]:
embeddings_df = embeddings_df.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
embeddings_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

Add unique `row_id` to each row in our main pyspark dataframe `df`

In [11]:
df = df.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
df.show()

+----------+--------------------+--------------------+--------------------+------+
|product_id|               title|         description|       combined_text|row_id|
+----------+--------------------+--------------------+--------------------+------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|     0|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|     1|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|     2|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|     3|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|     4|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|     5|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|     6|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|     7|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|     8|
|   

Let's join the two dataframes

In [12]:
df = df.join(embeddings_df, on="row_id", how="inner").drop("row_id")
df.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

### Cluster products using K-means

Assembling all the Embedding Columns into a Single 'features' Column

In [13]:
assembler = VectorAssembler(inputCols=features_column_names, outputCol="features")
data = assembler.transform(df)
data = data.select("product_id","title","description", "features")
data.show()

+----------+--------------------+--------------------+--------------------+
|product_id|               title|         description|            features|
+----------+--------------------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.0383744910359...|
|        P1|Turmode 30 ft. RP...|If you need more ...|[-0.1224376112222...|
|        P2|Large Tapestry Bo...|Polyester cover r...|[-0.0017069645691...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0025359764695...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[-0.0597195886075...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[-0.0525166280567...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[-0.0843156948685...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|[-0.1192158535122...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|[0.00212462316267...|
|        P9|Traditional Silve...|This transitional...|[-0.0048766154795...|
|       P10|

Apply K-Means Clustering with 5 Clusters on the `features` Column

In [14]:
kmeans = KMeans(k=5, featuresCol='features', predictionCol="cluster")
model = kmeans.fit(data)
clustered_data = model.transform(data)
clustered_data.show()

+----------+--------------------+--------------------+--------------------+-------+
|product_id|               title|         description|            features|cluster|
+----------+--------------------+--------------------+--------------------+-------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.0383744910359...|      4|
|        P1|Turmode 30 ft. RP...|If you need more ...|[-0.1224376112222...|      3|
|        P2|Large Tapestry Bo...|Polyester cover r...|[-0.0017069645691...|      4|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0025359764695...|      1|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[-0.0597195886075...|      3|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[-0.0525166280567...|      2|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[-0.0843156948685...|      0|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|[-0.1192158535122...|      3|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|[0.00212462316267...| 

### Visualizing the clusters

Let's reduce the dimensionality of our features for visualization purpose

`512 dimensions => 2 dimensions`

In [15]:
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(clustered_data)
pca_results = pca_model.transform(clustered_data)
pca_results.show()

+----------+--------------------+--------------------+--------------------+-------+--------------------+
|product_id|               title|         description|            features|cluster|        pca_features|
+----------+--------------------+--------------------+--------------------+-------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.0383744910359...|      4|[0.12741762580445...|
|        P1|Turmode 30 ft. RP...|If you need more ...|[-0.1224376112222...|      3|[-0.0204833377673...|
|        P2|Large Tapestry Bo...|Polyester cover r...|[-0.0017069645691...|      4|[0.14053662304658...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.0025359764695...|      1|[0.01842329250349...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[-0.0597195886075...|      3|[-0.1280724488471...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[-0.0525166280567...|      2|[0.07239008854289...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[

In [16]:
pca_df = pca_results.select(["product_id", "pca_features", "cluster"]).toPandas()
pca_df['x']=pca_df['pca_features'].apply(lambda x: x[0])
pca_df['y']=pca_df['pca_features'].apply(lambda x: x[1])
pca_df.head()

,product_id,pca_features,cluster,x,y
0,P0,"[0.12741762580445404, -0.25236054925432555]",4,0.127418,-0.252361
1,P1,"[-0.020483337767349598, 0.05521243426692503]",3,-0.020483,0.055212
2,P2,"[0.14053662304658748, -0.4206834622098323]",4,0.140537,-0.420683
3,P3,"[0.018423292503493345, 0.10781809224065278]",1,0.018423,0.107818
4,P4,"[-0.12807244884718053, -0.16229421302323502]",3,-0.128072,-0.162294


Let's plot the Clusters

In [17]:
def plot_clusters(pca_df, num_clusters=5):
    """
    Plots a 2D visualization of clusters using Plotly Express.

    Parameters:
    - pca_df (DataFrame): A Pandas DataFrame containing columns 'x', 'y', and 'cluster'.
      'x' and 'y' are the 2D PCA components, and 'cluster' indicates the cluster label.
    - num_clusters (int): The number of unique clusters to display.
    - recently_viewed_df (DataFrame, optional): DataFrame with 'x' and 'y' coordinates for recently viewed products.

    This function creates an interactive scatter plot where each point is colored according to its cluster.
    Recently viewed products are marked as black crosses if provided.

    Returns:
    - fig (Figure): The Plotly figure object for the plot.
    """

    # Create the base cluster plot
    fig = px.scatter(
        pca_df,
        x='x',
        y='y',
        opacity=0.6,
        size_max=4,
        color= pca_df.cluster.astype(str),
        title='2D Visualization of Clusters with Recently Viewed Products',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'},
        category_orders={'cluster': list(range(num_clusters))},
        # show the product id in the tooltip
        hover_data={'product_id': True}

    )

    # Update layout to add legend title and adjust plot settings
    fig.update_layout(legend_title_text='Clusters', legend=dict(x=1, y=1), width=600, height=500)

    return fig

fig = plot_clusters(pca_df)
fig.show()

### Highlight recently viewed products

In [18]:
print("The user has recently viewed the following products: ", recently_viewed_products)

The user has recently viewed the following products:  ['P316', 'P333', 'P1115', 'P1691', 'P1082', 'P397', 'P1441', 'P1054']


Let's have a look at the records in our `clustered_data` dataframe related to the recently viewed products.

In [23]:
filtered_data = clustered_data.where(F.col("product_id").isin(recently_viewed_products))
unique_clusters = filtered_data.select("cluster").distinct().rdd.flatMap(lambda x:x).collect()
print(unique_clusters)
filtered_data.show()

[4, 0]
+----------+--------------------+--------------------+--------------------+-------+
|product_id|               title|         description|            features|cluster|
+----------+--------------------+--------------------+--------------------+-------+
|      P316|Mystic Fitz Roy B...|With its distress...|[0.02460712008178...|      4|
|      P333|Florida Shag Beig...|Lavish natural mo...|[0.04508785530924...|      4|
|      P397|1 gal. #M250-3 Ap...|BEHR ULTRA SCUFF ...|[-0.0400431267917...|      0|
|     P1054|1 gal. #HDPG60 Mi...|The improved PPG ...|[-0.0941969677805...|      0|
|     P1082|1 qt. #S220-7 Mol...|BEHR ULTRA SCUFF ...|[-0.0792921781539...|      0|
|     P1115|Modern Gray/Multi...|This Modern Gray/...|[0.08428560942411...|      4|
|     P1441|1 qt. #PPU6-06 Ho...|BEHR PREMIUM PLUS...|[-0.0182529091835...|      0|
|     P1691|Genet Rust/Red-Br...|Add a refreshing ...|[-0.0520065464079...|      4|
+----------+--------------------+--------------------+---------------

### Recommend products based on recently viewed products

Let's have a look at the recently viewed products titles

In [22]:
filtered_data.select("title").rdd.flatMap(lambda x:x).collect()

["Mystic Fitz Roy Beige 9' 0 x 12' 0 Area Rug",
 'Florida Shag Beige/Multi 3 ft. x 5 ft. Floral Area Rug',
 '1 gal. #M250-3 Apple Turnover Extra Durable Flat Interior Paint & Primer',
 '1 gal. #HDPG60 Misty Emerald Lake Flat Interior Paint and Primer',
 '1 qt. #S220-7 Molasses Extra Durable Flat Interior Paint & Primer',
 'Modern Gray/Multi 9 ft. x 12 ft. Vibrant Abstract Polyester Area Rug',
 '1 qt. #PPU6-06 Honey Locust Eggshell Enamel Low Odor Interior Paint & Primer',
 'Genet Rust/Red-Brown 8 ft. x 11 ft. Abstract Wool Area Rug']

Let's see the distinct clusters of the recenetly viewed products.

In [24]:
print(unique_clusters)

[4, 0]


Let's find the possible products for the recommendation.

In [27]:
possible_recommendations=clustered_data.filter(clustered_data['cluster'].isin(unique_clusters)).filter(~clustered_data['product_id'].isin(recently_viewed_products))

Let's perform a groupby and generate a list of product IDs that can be recommended for each of the clusters.

In [30]:
recommendations = possible_recommendations.groupBy("cluster").agg(F.collect_list("product_id").alias("recommendations"))
recommendations_df=recommendations.toPandas()
recommendations_df['random_recommendations']=recommendations_df['recommendations'].apply(lambda x: np.random.choice(x,5,replace=False).tolist())
recommendations_df.head()

,cluster,recommendations,random_recommendations
0,4,"[P0, P2, P12, P21, P49, P52, P60, P71, P73, P8...","[P1544, P302, P457, P595, P481]"
1,0,"[P6, P11, P16, P18, P24, P26, P30, P33, P40, P...","[P545, P1306, P1378, P1313, P868]"


In [31]:
# write a python function to display the recommendations
def display_recommendations(row):
  # find the title of the product in df
  product_ids = row['random_recommendations']
  cluster = row.cluster

  titles = data. \
          filter(data["product_id"]. \
          isin(product_ids)).select("title").collect()

  print("\n")
  print("Recommendations for Cluster:", cluster)
  for title in titles:
    print(title[0])

recommendations_df.apply(display_recommendations, axis=1)



Recommendations for Cluster: 4
Sea Isle Blue 9 ft. x 12 ft. Area Rug
Tan/Rear FloorLiner/Land Rover/Range Rover/2007 - 2009/
14 in. x 21 in. Multicolor Australian Terrier Dish Drying Mat
Ryu Dark Grey Cement Texture Wallpaper Sample
Miami Hexagon 2 - Piece Twin Size Multi Colored Comforter Set Set


Recommendations for Cluster: 0
1 gal. #530D-7 Grand Rapids Flat Exterior Paint & Primer
5 gal. #M500-4 Hemisphere Satin Enamel Exterior Paint & Primer
5 gal. #590D-6 Wickford Bay Flat Exterior Paint & Primer
1 qt. #PPU6-20 Eden Prairie Hi-Gloss Enamel Interior/Exterior Paint
5 gal. #480B-4 Shoreline Green Satin Enamel Exterior Paint & Primer


,0
0,None
1,None
